In [1]:
pip install keras-tuner

In [2]:
import tensorflow as tf
import keras_tuner
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
import matplotlib.pyplot as plt
import numpy as np

In [3]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

11490434/11490434 [==============================] - 2s 0us/step


In [4]:
def build_model(hp):
    model = tf.keras.Sequential()

    # Convolutional layers
    for i in range(hp.Int('conv_layers', 1, 5, default=3)):
        model.add(layers.Conv2D(hp.Int(f'filters_{i}', 32, 256, step=32), (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Dropout(hp.Float(f'dropout_conv_{i}', 0, 0.5, step=0.1, default=0.25)))

    model.add(layers.Flatten())

    # Dense layers
    for i in range(hp.Int('dense_layers', 1, 5, default=3)):
        model.add(layers.Dense(hp.Int(f'neurons_{i}', 32, 256, step=32), activation='relu'))
        model.add(layers.Dropout(hp.Float(f'dropout_dense_{i}', 0, 0.5, step=0.1, default=0.25)))

    model.add(layers.Dense(10, activation='softmax'))

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', 1e-5, 1e-3, sampling='log', default=1e-3)),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [5]:
# Set up the Keras Tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='tune_dir',
    project_name='mnist_tuning'
)

Reloading Tuner from tune_dir/mnist_tuning/tuner0.json


In [6]:
# Run the tuner
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

In [7]:
tuner.results_summary(1)

Results summary
Results in tune_dir/mnist_tuning
Showing 1 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
conv_layers: 3
filters_0: 128
dropout_conv_0: 0.2
filters_1: 64
dropout_conv_1: 0.4
filters_2: 256
dropout_conv_2: 0.0
dense_layers: 4
neurons_0: 256
dropout_dense_0: 0.30000000000000004
neurons_1: 256
dropout_dense_1: 0.2
neurons_2: 160
dropout_dense_2: 0.2
learning_rate: 0.0006291832312051203
filters_3: 224
dropout_conv_3: 0.0
neurons_3: 256
dropout_dense_3: 0.30000000000000004
Score: 0.8773000240325928


In [8]:
# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]
best_model.build((None,) + x_train.shape[1:])
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 128)       1280      
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 128)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 13, 13, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        73792     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 5, 5, 64)          0

In [9]:
# Train the best model
fit = best_model.fit(
    x_train,y_train,
    validation_data=(x_test,y_test),
    epochs=20,batch_size=128,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=3)]
)

Epoch 1/20
469/469 [==============================] - 18s 22ms/step - loss: 0.6313 - accuracy: 0.8060 - val_loss: 0.1143 - val_accuracy: 0.9675
Epoch 2/20
469/469 [==============================] - 6s 12ms/step - loss: 0.1799 - accuracy: 0.9510 - val_loss: 0.0687 - val_accuracy: 0.9806
Epoch 3/20
469/469 [==============================] - 6s 13ms/step - loss: 0.1366 - accuracy: 0.9628 - val_loss: 0.0600 - val_accuracy: 0.9828
Epoch 4/20
469/469 [==============================] - 5s 12ms/step - loss: 0.1078 - accuracy: 0.9707 - val_loss: 0.0566 - val_accuracy: 0.9843
Epoch 5/20
469/469 [==============================] - 6s 12ms/step - loss: 0.0940 - accuracy: 0.9747 - val_loss: 0.0535 - val_accuracy: 0.9851
Epoch 6/20
469/469 [==============================] - 6s 12ms/step - loss: 0.0858 - accuracy: 0.9759 - val_loss: 0.0504 - val_accuracy: 0.9859
Epoch 7/20
469/469 [==============================] - 5s 11ms/step - loss: 0.0787 - accuracy: 0.9782 - val_loss: 0.0463 - val_accuracy: 0.986

In [10]:
# Evaluate the model on the test set
loss, accuracy = best_model.evaluate(x_test, y_test)
print(f'Test accuracy: {accuracy * 100:.2f}%')

313/313 [==============================] - 1s 3ms/step - loss: 0.0402 - accuracy: 0.9898
Test accuracy: 98.98%
